### Adaptive Derivative-Assembled Pseudo-Trotter ansatz Variational Quantum Eigensolver [1] (ADAPT-VQE)

The Unitary Coupled Clusters truncated at the level of double excitations (UCCSD)
$$ |\psi^{UCCSD}\rangle \equiv e^{T(1)+T(2)}|\psi^{HF}\rangle, \label{eq:1}\tag{1}$$
where
$$ T(1)=\sum_{ia} \hat \tau_{ia} = \sum_{ia} \tau_{ia} (a^{\dagger}_{a} a_{i} - a^{\dagger}_{i} a_{a}),$$
$$T(2) = \sum_{iajb} \hat \tau_{iajb} = \sum_{iajb} \tau_{iajb} (a^{\dagger}_{a}  a^{\dagger}_{b} a_{j} a_{i} - a^{\dagger}_{i} a^{\dagger}_{j} a_{b} a_{a}).$$
is a natural ansatz for Variational Quantum Eigensolver since it is parametrized by the unitary operators and has a polynomial number of variational parameters. For the practical application on the quantum computer, it is desired to decompose unitary operator in Eq. (\ref{eq:1}) into a product of unitary operators acting on a few qubits. This can be achieved via the first-order Trotter decomposition. Because of the variational flexibility of the ansatz (parameters are optimized after the Trotterization), one can sufficiently reproduce the UCCSD results performing only single Trotter step
$$ |\psi^{UCCSD}\rangle \approx \prod_{ia}e^{\hat \tau_{ia}} \prod_{iajb}e^{\hat \tau_{iajb}} |\psi^{HF}\rangle. \label{eq:2}\tag{2}$$
The ADAPT-VQE algorithm suggests limiting the number of unitary operators in Eq. (\ref{eq:2}) to lower the depth of the circuit and make ansatz even more practical for quantum computing. This is acquired by the systematic extension of the ansatz by one excitation operator at a time starting from the Hartree Fock state. Specifically, having specified the pool of all possible single and double excitation operators
$\{ \hat A_1, \hat A_2, \dots, \hat A_n \}$ ($\hat A_i \in \{ a^{\dagger}_a a_i - a^{\dagger}_i a_a; a^{\dagger}_{a}  a^{\dagger}_{b} a_{j} a_{i} - a^{\dagger}_{i} a^{\dagger}_{j} a_{b} a_{a} \}$), at first one evaluates the gradient $$\frac{\partial E}{\partial \theta_m} = 
\left.\frac{\partial}{\partial \theta_m} \langle \psi^{HF} |e^{-\theta_1 \hat A_1} e^{-\theta_2 \hat A_2} \ldots
 \hat H \ldots e^{\theta_2 \hat A_2} e^{\theta_1 \hat A_1}| \psi^{HF} \rangle \right|_{ \forall \theta_i=0} 
 =\langle \psi^{HF} | [ \hat H,\hat A_m] | \psi^{HF} \rangle . \label{eq:3}\tag{3}$$
 Then ansatz is extended with the excitation operator,$\hat A_k$, corresponding to the highest element of the gradient. Afterward, all or only the new variational parameters are optimized. Continuing this process until the energy, the L$^2$ norm of the gradient or the maximum element is converged one recovers ADAPT-VQE[1]
$$|\psi^{ADAPT}\rangle \equiv (e^{\hat \tau_N}) (e^{\hat \tau_{N-1}}) \ldots (e^{\hat \tau_2}) (e^{\hat \tau_1}) |\psi^{HF}\rangle, \label{eq:4}\tag{4}$$
where $\hat \tau_p$ can be either $\hat \tau_{ia}$ or $\hat \tau_{iajb}$. It should be emphasized that operator's  pool is never drained and always contains the all possible single and double excitation operators, thus allowing to some excitation operator,$\hat A_k$, enter the ansatz multiple times.

[1] Harper R. Grimsley, Sophia E. Economou, Edwin Barnes & Nicholas J. Mayhall, *Natur. Commun.* 10, 3007 (2019)

In [1]:
from qiskit import Aer
#import warnings
#warnings.filterwarnings('ignore')

# setup qiskit_chemistry logging
import logging
from qiskit.chemistry import set_qiskit_chemistry_logging
set_qiskit_chemistry_logging(logging.ERROR) # choose among DEBUG, INFO, WARNING, ERROR, CRITICAL and NOTSET

# chemistry related modules
from qiskit.chemistry import FermionicOperator
from qiskit.chemistry.drivers import PySCFDriver, UnitsType

#setup token for calculation on real device
# from qiskit import IBMQ
# provider = IBMQ.load_account()

### Molecule Setup

The hydrogen molecule (H$_2$) at the equilibrium bond length (0.735 Angstrom) is studied in this tutorial. In order to set up the molecular Hamiltonian one have to get the one- and two-electron integrals from a computational chemistry driver (PySCF in this case) employing the Hartree Fock calculation in STO-3G basis set. The information from the driver is saved to the ``molecule`` object. Then the Fermionic Hamiltonian is mapped to the qubit Hamiltonian using the ``parity`` mapping type as it further reduces the problem size.

In [2]:
# Use PySCF, a classical computational chemistry software
# package, to compute the one-body and two-body integrals in
# molecular-orbital basis, necessary to form the Fermionic operator

# chemistry related modules
from qiskit.chemistry import FermionicOperator
from qiskit.chemistry.drivers import PySCFDriver, UnitsType

driver = PySCFDriver(atom='H .0 .0 .0; H .0 .0 0.735',
                     unit=UnitsType.ANGSTROM,
                     basis='sto3g')
molecule = driver.run()
num_particles = molecule.num_alpha + molecule.num_beta
num_spin_orbitals = molecule.num_orbitals * 2

from qiskit.aqua.operators.weighted_pauli_operator import Z2Symmetries
from qiskit.aqua.operators.op_converter import to_weighted_pauli_operator
ferOp = FermionicOperator(h1=molecule.one_body_integrals, h2=molecule.two_body_integrals)

# Build the qubit operator, which is the input to the VQE algorithm in Aqua
map_type = 'PARITY'
qubitOp = ferOp.mapping(map_type)
# adapted this to work with master branch!
qubitOp = Z2Symmetries.two_qubit_reduction(to_weighted_pauli_operator(qubitOp), num_particles)
num_qubits = qubitOp.num_qubits

### Initial State and Optimizer
Set the initial state to the Hartree Fock state, $| \psi^{HF} \rangle$, and choose the optimizer for the VQE algorithm (in this case, limited memory BFGS from scipy).

In [3]:
# setup the initial state for the variational form
from qiskit.chemistry.aqua_extensions.components.initial_states import HartreeFock
init_state = HartreeFock(num_qubits, num_spin_orbitals, num_particles)

# set the backend for the quantum computation
from qiskit import Aer
backend = Aer.get_backend('statevector_simulator')

# setup a classical optimizer for VQE
from qiskit.aqua.components.optimizers import L_BFGS_B
optimizer = L_BFGS_B()

### UCCSD-VQE reference
Perform the reference calculation employing ``UCCSD`` variational form and ``VQE`` algorithm.

In [13]:
# setup the variational form for VQE
from qiskit.chemistry.aqua_extensions.components.variational_forms import UCCSD
var_form = UCCSD(num_qubits, 1, num_spin_orbitals, num_particles, initial_state=init_state)
from qiskit.aqua.algorithms.adaptive import VQE
algorithm = VQE(qubitOp, var_form, optimizer)
result = algorithm.run(backend)

### ADAPT-VQE run
Now instantiate the ``UCCSDAdapt`` object and pass it to ``VQEadapt``, specially developed for the sequential extension and optimization of the ADAPT-VQE ansatz. In contrast to ``VQE`` one have to additionally pass ``threshold``and ``delta`` parameters to the ``VQEAdapt`` constructor. The  former defines the stopping criteria for the ansatz extension setting the threshold for the maximum element of the gradient, the latter sets the accuracy of the gradient evaluation. In current ADAPT-VQE implementation, the gradient is evaluated by the central difference approximation. Also one has to supply the pool of excitation operators, which is initialized to all possible single and double excitations in ``UCCSDAdapt``.

In [14]:
from qiskit.chemistry.aqua_extensions.components.variational_forms import UCCSDAdapt
var_form_base = UCCSDAdapt(num_qubits, 1, num_spin_orbitals, num_particles, initial_state=init_state)

from qiskit.chemistry.aqua_extensions.algorithms.adaptive import VQEAdapt
algorithm = VQEAdapt(qubitOp, var_form_base, optimizer, var_form_base.excitation_pool, threshold=0.00001, delta=0.1)
result_adapt = algorithm.run(backend)

### Compare results
Print and compare the gates utilized in both parametrizations and corresponding energies.

In [15]:
from prettytable import PrettyTable
table = PrettyTable()
table.field_names = ["Ansatz","Energy (Hartree)","Gates"]
cirq = var_form.construct_circuit(result['opt_params'])
table.add_row(['UCCSD',str(result['energy']),cirq.count_ops()])

cirq_adapt = var_form_base.construct_circuit(result_adapt['opt_params'])
table.add_row(['ADAPT-VQE', str(result_adapt['energy']), cirq_adapt.count_ops()])
print(table)

+-----------+---------------------+----------------------------------------------------+
|   Ansatz  |   Energy (Hartree)  |                       Gates                        |
+-----------+---------------------+----------------------------------------------------+
|   UCCSD   | -1.8572750302023784 | {'u3': 9, 'u1': 4, 'barrier': 3, 'u2': 4, 'cx': 4} |
| ADAPT-VQE | -1.8572750302023793 | {'u3': 5, 'u2': 4, 'cx': 4, 'u1': 2, 'barrier': 1} |
+-----------+---------------------+----------------------------------------------------+
